# 读取excel

In [3]:
import pandas as pd

df=pd.read_excel('./task-data/灾害任务数据.xlsx',sheet_name="Sheet1")
df.head(3)

,任务ID,任务名称,任务目标描述,hasDevelopmentStage,withScale,needData,useMethod,useResource,Unnamed: 8
0,Task_1_3_dailyRainfall,诊断期-城市群-日降雨量（mm）,"日/次,10-100km",mitigation,Citycluster,"Data_EC6392,Data_QO9144,",NaN,"GMI_GPM,DPR,DCS_2",NaN
1,Task_1_2_dailyRainfall,诊断期-城市-日降雨量（mm）,"日/次,10-30km",mitigation,City,"Data_EC6392,Data_QO9144,Data_LK4061",NaN,"GMI_GPM,DPR,DCS_2",NaN
2,Task_1_1_dailyRainfall,诊断期-街区-日降雨量（mm）,"日/次,10-100m",mitigation,Block,"Data_LK4061,Data_AV3010",NaN,"GMI_GPM,DPR,DCS_2",NaN


# 新建一个ttl文件并写入前缀


In [2]:
# 创建一个ttl文件，文件名为filename,并向文件写入prefix
def ttl_create(filename, prefix):
    relative_path = "./rdf/"  # 新创建的txt文件的存放路径
    full_path = relative_path + filename + '.ttl'  
    file = open(full_path, 'w')
    file.write(prefix)   #msg也就是下面的Hello world!
    file.close()

name="观测任务实例"
prefix='''@prefix : <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schema.org/> .
@base <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology> .

'''

ttl_create(name, prefix)


# 创建实例


In [6]:
def create_taskIndividual(df):
    text=""
    for index,row in df.iterrows():
        identifier=row["任务ID"]
        name=row["任务名称"]
        description=row["任务目标描述"]
        stage=row['hasDevelopmentStage']
        scale=row['withScale']

        needData=row['needData']
        useMethod=row['useMethod']
        useResource=row['useResource']

        task_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
:{identifier} rdf:type owl:NamedIndividual , :ObservationTask ;
:taskDescription "{description}" ;
:taskIdentifier "{identifier}" ;
:dataName "{name}" ;
:hasDevelopmentStage :{stage} ;
:withScale :{scale} ;
rdfs:label "{name}" .
'''

        if not pd.isnull(needData):
            needData=needData.replace(",",", :")
            task_text=task_text+f'''
:{identifier} :needData :{needData}.'''

        if not pd.isnull(useMethod):
            useMethod=useMethod.replace(",",", :")
            task_text=task_text+f'''
:{identifier} :useMethod :{useMethod}.'''
        
        if not pd.isnull(useResource):
            useResource=useResource.replace(",",", :")
            task_text=task_text+f'''
:{identifier} :useObsResource :{useResource} .'''

        text=text+task_text
    return text
        
text = create_taskIndividual(df)    
file = open("./rdf/观测任务实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()